In [ ]:
# 01_CAPTURE_DRIVE.ipynb
# %% [markdown]
"""
# 01_CAPTURE_DRIVE: Captura de Arquivos do Google Drive e Envio ao MinIO

Este notebook copia arquivos do Google Drive (remote `gdrive`) para um diretório temporário
no container e então os envia ao bucket `raw-data` no MinIO, sob o prefixo `projeto-mba/`.
"""
# %% [code]
import os
import tempfile
import subprocess
from minio import Minio

# %% [code]
# 1) Criar diretório temporário para download
tmp_dir = tempfile.mkdtemp(prefix="projeto_mba_")
print(f"Diretório temporário criado: {tmp_dir}")

# %% [code]
# 2) Sincronizar arquivos do Google Drive (remote 'gdrive') para o temp
subprocess.run([
    "rclone", "copy", "gdrive:", tmp_dir,
    "--create-empty-src-dirs"
], check=True)
print("Arquivos copiados do Google Drive para o diretório temporário.")

# %% [code]
# 3) Configurar cliente MinIO
minio_client = Minio(
    "minio:9000",
    access_key=os.getenv("MINIO_ROOT_USER", "admin"),
    secret_key=os.getenv("MINIO_ROOT_PASSWORD", "senhasegura"),
    secure=False
)
bucket = "raw-data"
if not minio_client.bucket_exists(bucket):
    minio_client.make_bucket(bucket)
    print(f"Bucket criado: {bucket}")
else:
    print(f"Bucket existente: {bucket}")

# %% [code]
# 4) Enviar arquivos para o bucket 'raw-data/projeto-mba/'
for root, _, files in os.walk(tmp_dir):
    for file in files:
        full_path = os.path.join(root, file)
        rel_path = os.path.relpath(full_path, tmp_dir)
        object_name = f"projeto-mba/{rel_path}"
        minio_client.fput_object(bucket, object_name, full_path)
        print(f"Uploaded: {object_name}")

# %% [markdown]
"""
**Conclusão**: Os arquivos de "Projeto MBA" do Google Drive foram carregados no bucket `raw-data/projeto-mba/` e estão prontos para a etapa de ingestão (raw.data_files).
"""
